# Compute Energies of Displacements Along Vibrational Models
Compute the vibrational modes using a lower level of theory then displace along those axes. Options include:
- Energy scale of vibrations. Set the anticipated energy increase based on eigenvalue of the Hessians
- Number of modes to vibrate at a time.
- Maximum length of displacement vector.

In [ ]:
from jitterbug.utils import make_calculator
from ase.vibrations import Vibrations
from ase.io import write, read
from ase.db import connect
from ase import Atoms, units
from pathlib import Path
from tqdm import tqdm 
import numpy as np
import os

Configuration

In [ ]:
starting_geometry = '../data/exact/caffeine_pm7_None.xyz'
threads = min(os.cpu_count(), 12)
step_size: float = 0.002 # Target energy increase (units: eV)
perturbs_per_evaluation: int = 16  # Number of perturbations to perform at once
max_perturb = 0.06  # Maximum length of displacement vector
lower_level: tuple[str, str] = ('xtb', None)

Derived

In [ ]:
run_name = Path(starting_geometry).name[:-4]
name, method, basis = run_name.split("_")

## Load in the Relaxed Structure
We generated a relaxed structure in the previous notebook

In [ ]:
atoms = read(starting_geometry)
atoms

## Assemble the List of Perturbations
Start by computing the vibrational modes using a lower level of theory, then pick the vibrational modes that are large enough

In [ ]:
lower_calc = make_calculator(*lower_level)

Compute the vibrational modes

In [ ]:
%%time
atoms.calc = lower_calc
vib = Vibrations(atoms)
vib.run()

In [ ]:
vib_data = vib.get_vibrations()

Get the eigenvalues and eigenvectors.

In [ ]:
evalues, emodes = np.linalg.eigh(vib_data.get_hessian_2d())

Remove the six smallest eignvalues, which should be zero

In [ ]:
evalues = evalues[6:]
emodes = emodes[6:, :]

Scale the eigenmodes such that a perturbation of length 1 should increase the energy by the target `step_size`.

The value of the eigenmode is the a second derivative wrt atom positions. So, $\Delta E = 0.5 \lambda_i \delta x_i^2$ and $x_i = \sqrt{2 \Delta E / \lambda_i}$


In [ ]:
scale_mag = np.clip(np.sign(evalues) * np.sqrt(2 * step_size / np.abs(evalues)),
                    a_min=-max_perturb,
                    a_max=max_perturb)

In [ ]:
scaled_modes = emodes * scale_mag[:, None]

Check my math

In [ ]:
perturbed_atoms = atoms.copy()
perturbed_atoms.positions += scaled_modes[0, :].reshape((-1, 3))

In [ ]:
perturbed_energy = lower_calc.get_potential_energy(perturbed_atoms) - lower_calc.get_potential_energy(atoms)

## Run the Perturbations
Run as many as we should need to fill all degrees of freedom for the Hessian.

In [ ]:
n_atoms = len(atoms)
to_compute = 3 * n_atoms + 3 * n_atoms * (3 * n_atoms + 1) // 2 + 1
print(f'Need to run {to_compute} calculations for full accuracy.')

Reduce the number of perturbations if too large

In [ ]:
perturbs_per_evaluation = min(scaled_modes.shape[0], perturbs_per_evaluation)

Prepare the output directory

In [ ]:
out_dir = Path('data') / 'along-vibrational-modes'
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
db_path = out_dir / f'{run_name}_d={step_size:.2e}-N={perturbs_per_evaluation}-maxstep={max_perturb:.2e}-lower={"+".join(map(str, lower_level))}.db'
print(f'Writing to {db_path}')

Make the calculator

In [ ]:
calc = make_calculator(method, basis, num_threads=threads)
atoms.calc = calc

Add the relaxed geometry if needed

In [ ]:
if not db_path.is_file():
    atoms.get_potential_energy()
    with connect(db_path) as db:
        db.write(atoms, name='initial')

Generate the energies

In [ ]:
with connect(db_path) as db:
    num_done = len(db) - 1
print(f'We have finished {num_done} perturbations already. {to_compute - num_done} left to do.')

In [ ]:
rng = np.random.RandomState(1)
iterator = tqdm(range(to_compute - num_done))
for i in iterator:
    # Choose a number of perturbation vectors
    to_disp = rng.choice(scaled_modes.shape[0], size=(perturbs_per_evaluation,), replace=False)
    
    # Pick a random magnitude for each
    disp = (scaled_modes[to_disp, :] * rng.uniform(-1, 1, size=(perturbs_per_evaluation, 1))).sum(axis=0)
    
    # Make the new atoms
    new_atoms = atoms.copy()
    new_atoms.positions += disp.reshape((-1, 3))
    
    # Make the name for the computation
    name = f"perturb-{i}_modes-{','.join(map(str, to_disp))}"
    iterator.set_description(name)
    
    # Compute the energy and store in the db
    new_atoms.calc = calc
    new_atoms.get_potential_energy()
    with connect(db_path) as db:
        db.write(new_atoms, name=name)